In [422]:
import rlcard
from rlcard.envs import make
from rlcard.games.nolimitholdem import Judger
from rlcard.games.limitholdem.utils import Hand
import numpy as np

In [423]:
#Bot fishplayer
#Always calls regardless of the hand


def fishplayer(state):

    action = state['raw_legal_actions'][1]

    return action

# Bot nitplayer
#Only plays if he hase either an Ace or a combination of Kings, Queens, Jacks or Tens otherwise he folds

def nitplayer(state):

    nitcard = {'K', 'Q', 'J', 'T'}

    card1 = state['raw_obs']['hand'][1][1]
    card2 = state['raw_obs']['hand'][0][1]

    if 'A' in card1 or 'A' in card2:
        try:
            action = state['raw_legal_actions'][3]
        except:
            action = state['raw_legal_actions'][1]

    elif (card1 in nitcard) and (card2 in nitcard) :
        try:
            action = state['raw_legal_actions'][3]
        except:
            action = state['raw_legal_actions'][1]     

    else:
        action = state['raw_legal_actions'][0]

    return action

# Bot GTOpreflopbot
# Plays according to GTO strategy

def GTOplayer(state):


    bet4range = {'AA','AK','AK'}
    bet3range = {'AQ','QQ','JJ'}
    callrange = ['2A', '3A', '4A', '5A', '6A', '7A', '8A', '9A', '22', '33', '44', '55', '66', '77', '88', '99', '9J', '9K', '9Q', 'AJ', 'AT', 'JQ', 'JK', 'JT', 'KQ', 'QT']

    card1 = state['raw_obs']['hand'][1][1]
    card2 = state['raw_obs']['hand'][0][1]

    hand = ''.join(sorted(card1 + card2))

    if hand in bet4range:
        try:
            action = state['raw_legal_actions'][4]
        except:
            action = state['raw_legal_actions'][1]

    elif hand in bet3range:
        try:
            action = state['raw_legal_actions'][3]
        except:
            action = state['raw_legal_actions'][1]

    elif hand in callrange:
        action = state['raw_legal_actions'][1]

    else:
        action = state['raw_legal_actions'][0]
    
    return action


# playerorchestrator

def playerorchestrator(state):
    activeplayer = playerdict[state['raw_obs']['current_player']]
    a = activeplayer(state)
    return a

In [ ]:
playerdict = {
    0: fishplayer,
    1: nitplayer,
    2: GTOplayer
}

env = make('no-limit-holdem', config={'game_num_players': 3})
payoffs = np.zeros(3)

for i in range(10000):

    state, player_id = env.reset()

    done = False    while not done:

        action = playerorchestrator(state)

        #print(f"Player {player_id} chooses: {action} with hand {state['raw_obs']['hand']} and public cards {state['raw_obs']['public_cards']}")
        state, player_id = env.step(action)
        done = env.is_over()
        #if done:
            #print(state['raw_obs']['public_cards'])

    # Get final payoffs
    #print("Payoffs:", env.get_payoffs())
    payoffs = payoffs + env.get_payoffs()

print("Player 0: Fishplayer, Payoff: ", payoffs[0])
print("Player 1: Nitplayer, Payoff: ", payoffs[1])
print("Player 2: GTOplayer, Payoff: ", payoffs[2])

Player 0: Fishplayer, Payoff:  -32108.0
Player 1: Nitplayer, Payoff:  31958.0
Player 2: GTOplayer, Payoff:  150.0
